## Partitions when Unioning or Binding DataFrames (working title)

You can append two DataFrames in PySpark that have the same schema with the [`.union()`](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.union) operation. In sparklyr, the equivalent operation is [`sdf_bind_rows()`](https://spark.rstudio.com/reference/sdf_bind.html) and R users will often refer to appending two DataFrames as *binding*.

The `.union()` function in PySpark and the `sdf_bind_rows()` function in sparklyr are both equivalent to `UNION ALL` in SQL. A regular `UNION` operation in SQL will remove duplicates, whereas `.union()` in PySpark and `sdf_bind_rows()` in sparklyr will not.

This article uses the PySpark operation `.union()` as an example, but the principles are identical in sparklyr when using `sdf_bind_rows()`.

Unlike joining two DataFrames, `.union()` does not involve a full shuffle, as the data does not move between partitions. Instead, the number of partitions in the unioned DataFrame is equal to the sum of the number of partitions in the two source DataFrames, e.g. if you union a DataFrame consisting of 100 partitions and one consisting of 50 partitions, your unioned DataFrame will have 150 partitions.

### Section needed?
This issue was highlighed by a user who had created a function which split a DataFrame into three smaller DataFrames, did different calculations to each, and then unioned them together. Each time the function was called the number of partitions in the DataFrame was multiplied by three, so a DataFrame consisting of 200 partitions then had 600, then 1800 etc, and these excessive partitions caused the Spark session to crash, as excessive partitioning is inefficient. [Checkpoints and Staging Tables](https://gitlab-app-l-01/DAP_CATS/troubleshooting/tip-of-the-week/-/blob/master/tip_21_checkpoint.ipynb) or using [staging tables](https://gitlab-app-l-01/DAP_CATS/troubleshooting/tip-of-the-week/-/blob/master/tip_15_staging_tables.ipynb) to cut the lineage of the DataFrame did not work, as the data was written out to disk and then read back in with the same number of partitions.
### end

To avoid this issue, you can use [`.coalesce()`](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.coalesce) or [`.repartition()`](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.repartition) to reduce the number of partitions (use [`sdf_coalesce()`](https://spark.rstudio.com/reference/sdf_coalesce.html) or [`sdf_repartition()`](https://spark.rstudio.com/reference/sdf_repartition.html)
 in sparklyr). The DataFrame will also get repartitioned when a *wide transformation* is applied to the DataFrame (also called a shuffle), e.g. with a `.groupBy()` or `.orderBy()`.
 
It is also worth being aware that storing data on HDFS as many [small files](https://gitlab-app-l-01/DAP_CATS/troubleshooting/tip-of-the-week/-/blob/master/tip_54_coalesce_small_files.md) is inefficient, both in terms of how the data is stored and in reading it in. Unioning many DataFrames and then writing straight to HDFS can be a cause of this issue.
## An example

Start a Spark session, read the Animal Rescue data, group by animal and year, and then count. The grouping and aggregation will cause a shuffle, meaning that the DataFrame will have 12 partitions, as we set `spark.sql.shuffle.partitions` to `12` in the `SparkSession.builder`.


In [ ]:
from pyspark.sql import SparkSession, functions as F

spark = (
    SparkSession.builder.appName("festive-dates")
    .config("spark.executor.memory", "1g")
    .config("spark.executor.cores", 1)
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.maxExecutors", 3)
    .config("spark.sql.shuffle.partitions", 12)
    .config("spark.shuffle.service.enabled", "true")
    .config("spark.ui.showConsoleProgress", "false")
    .enableHiveSupport()
    .getOrCreate()
)

rescue = (spark.read.csv("/training/animal_rescue.csv", header=True, inferSchema=True)
          .withColumnRenamed("IncidentNumber", "incident_number")
          .withColumnRenamed("AnimalGroupParent", "animal_group")
          .withColumnRenamed("CalYear", "cal_year")
          .groupBy("animal_group", "cal_year")
          .agg(F.count("incident_number").alias("animal_count")))

rescue.limit(5).toPandas()

We can confirm the number of partitions with `.rdd.getNumPartitions()`:

In [ ]:
rescue.rdd.getNumPartitions()

Now create some smaller DataFrames, containing different animals, and preview one of them:

In [ ]:
dogs = rescue.filter(F.col("animal_group") == "Dog")
cats = rescue.filter(F.col("animal_group") == "Cat")
hamsters = rescue.filter(F.col("animal_group") == "Hamster")

dogs.limit(5).toPandas()

Each of these DataFrames has 12 partitions:

In [ ]:
print(dogs.rdd.getNumPartitions(),
      cats.rdd.getNumPartitions(),
      hamsters.rdd.getNumPartitions())

When we union two of them, we now get 12 + 12 = 24 partitions:

In [ ]:
dogs_and_cats = dogs.union(cats)
dogs_and_cats.rdd.getNumPartitions()

Unioning another DataFrame adds another 12 partitions to make 36 (24 + 12):

In [ ]:
dogs_cats_hamsters = dogs_and_cats.union(hamsters)
dogs_cats_hamsters.rdd.getNumPartitions()

Although we only have 36 partitions here it is easy to see how this might get excessive with too many `.union()` statements.

A subsequent shuffle (e.g. sorting the DataFrame) will reset the number of partitions to that specified in `spark.sql.shuffle.partitions`:

In [ ]:
dogs_cats_hamsters.orderBy("animal_group", "cal_year").rdd.getNumPartitions()


## **This is a reference to the work Nathan is working on**
You can also use `.repartition()` or `.coalesce()`. `.repartition()` involves a shuffle of the DataFrame and puts the data into roughly equal partition sizes, whereas `.coalesce()` combines partitions without a full shuffle, and so is more efficient, although at the potential cost of less equal partition sizes and therefore potential skew in the data. See the [Small files issue](https://gitlab-app-l-01/DAP_CATS/troubleshooting/tip-of-the-week/-/blob/master/tip_54_coalesce_small_files.md) for more information.



In [ ]:
dogs_cats_hamsters.repartition(20).rdd.getNumPartitions()


### Further Resources

Spark at the ONS Articles:
- [Checkpoints and Staging Tables](../spark-concepts/checkpoint-staging.md)

PySpark Documentation:
- [`.union()`](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.union)
- [`.coalesce()`](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.coalesce)
- [`.repartition()`](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.repartition)

sparklyr Documentation:
- [`sdf_bind_rows()`](https://spark.rstudio.com/reference/sdf_bind.html)
- [`sdf_coalesce()`](https://spark.rstudio.com/reference/sdf_coalesce.html)
- [`sdf_repartition()`](https://spark.rstudio.com/reference/sdf_repartition.html)

Spark SQL Documentation:
- [`round`](https://spark.apache.org/docs/latest/api/sql/index.html#round)
- [`bround`](https://spark.apache.org/docs/latest/api/sql/index.html#bround)